In [1]:
from elasticsearch import Elasticsearch
import json
from elasticsearch.helpers import scan
from elasticsearch_dsl import Search,Q
from credentials import usr_pwd

In [2]:
hosts = ["10.1.8.6:9200"]
user,pas = usr_pwd()

In [3]:
es = Elasticsearch(hosts,
                  timeout = 6000,
                  use_ssl = True,
                  verify_certs = False,
                  ssl_show_warn = False,
                  raise_on_error = False,
                  http_auth = (user,pas)
                  )

In [4]:
es.ping()

True

In [5]:
import pandas as pd
import numpy as np

/home/pavan_rs/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [6]:
df = pd.read_excel("New MD5 cost downs.xlsx")
df

,md5,From,To,Subject,Month/Year
0,a7551870dbe26c7b810c986c023f7ea6,NaN,NaN,NaN,NaN
1,629ac74d1f717403516b0852e43461ad,NaN,NaN,NaN,NaN
2,9a58f66ce99522ba8649112a143027e0,NaN,NaN,NaN,NaN
3,79baab175907a5c9525471edf8c298b8,NaN,NaN,NaN,NaN
4,962a4f55273c5358bb5100e64b9d2df5,NaN,NaN,NaN,NaN
...,...,...,...,...,...
4577,56039898dd0baa9c720b76983ed3097c,NaN,NaN,NaN,NaN
4578,b88f24c4c8d8c0b665c8ad945a602941,NaN,NaN,NaN,NaN
4579,ad912bda79550d4fca64331606ace7be,NaN,NaN,NaN,NaN
4580,1fe45e6333f07ea85abe67ae5762a66e,NaN,NaN,NaN,NaN


In [7]:
df_md5 = df["md5"].values.flatten()
df_md5[:10]

array(['a7551870dbe26c7b810c986c023f7ea6',
       '629ac74d1f717403516b0852e43461ad',
       '9a58f66ce99522ba8649112a143027e0',
       '79baab175907a5c9525471edf8c298b8',
       '962a4f55273c5358bb5100e64b9d2df5',
       '2cc1a91f80bbe2832936672cdf7855d6',
       '0b47d1c3f59e72c58aabb9e2140ac330',
       '850ebd0786d2a741db7abe7a4adcc2f0',
       'f8ba29c416f627f80fb32f71f16f4a2e',
       '2fa3975e6cbf3d1e4c8e79c37508193d'], dtype=object)

In [13]:
# query = {
#     "bool":{
#         "must":[{
#             "match_phrase_prefix":{
#                 "fileType":"Email"
#             }
#         },
#         {
#             "match_phrase":{
#                 "Attachment.MD5":"927bf1a54ca56ed2ac1fe247d70725c5"
#             }
#         }]
#     }
# }

# s = Search(using=es, index="*")
# s.query = Q(query)
# s = s.source(["content","metaData","Attachment"])

# for x in s.scan():
#     print(x.metaData["from"])

In [14]:
# i=0
# for val in df_md5:
#     i+=1
#     s = Search(using=es, index="*")
#     s.query = Q(fetch_md(val))
#     s = s.source(["content","metaData","Attachment"])
#     val_lis = []
#     frm = []
#     to = []
#     sub = []
#     date = []
#     for x in s.scan():
#         if "from" in x.metaData:
#             frm.append(x.metaData["from"])
#         if "to" in x.metaData:
#             to.append(x.metaData["to"])
#         if "subject" in x.metaData:
#             sub.append(x.metaData["subject"])
#         if "date" in x.metaData:
#             date.append(x.metaData["date"]) 
#     val_lis.extend([frm,to,sub,date])
#     if i==10:
#         break
#     print(val_lis)

In [15]:
def fetch_md(md5_val):
    query = {
    "bool":{
        "must":[{
            "match_phrase_prefix":{
                "fileType":"Email"
            }
        },
        {
            "match_phrase":{
                "Attachment.MD5":md5_val
                }
            }]
        }
        
     }
    return query

In [16]:
from collections import Iterable
def flatten(lis):
     for item in lis:
            if isinstance(item, Iterable) and not isinstance(item, str):
                for x in flatten(item):
                    yield x
            else:        
                yield item

In [19]:
import csv
header = ["MD5","FROM","TO","SUBJECT","DATE"]
with open("new_MD5_Costdown.csv","a") as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(header)

In [20]:
from tqdm import tqdm
import csv
i=0
val_lis = []
for val in tqdm(df_md5):
    i+=1
    s = Search(using=es, index="*")
    s.query = Q(fetch_md(val))
    s = s.source(["content","metaData","Attachment"])
    frm = []
    to = []
    sub = []
    date = []
    for x in s.scan():
        if "from" in x.metaData:
            frm.append(x.metaData["from"])
        if "to" in x.metaData:
            to.append(x.metaData["to"])
        if "subject" in x.metaData:
            sub.append(x.metaData["subject"])
        if "date" in x.metaData:
            date.append(x.metaData["date"])
    frm = list(flatten(frm))
    to = list(flatten(to))
    sub = list(flatten(sub))
    date = list(flatten(date))
    val_lis.extend([[val,set(frm),set(to),set(sub),set(date)]])
    if i==100:
        with open("new_MD5_Costdown.csv","a") as file:
            csvwriter = csv.writer(file)
            csvwriter.writerows(val_lis)
        val_lis=[]
        i=0
if i!=0:
    with open("new_MD5_Costdown.csv","a") as file:
            csvwriter = csv.writer(file)
            csvwriter.writerows(val_lis)

100%|██████████| 4582/4582 [1:14:58<00:00,  1.02it/s]


In [21]:
import pandas as pd
df_fin = pd.read_csv("new_MD5_Costdown.csv")
df_fin

,MD5,FROM,TO,SUBJECT,DATE
0,a7551870dbe26c7b810c986c023f7ea6,"{'Manisha.Joshi@cantire.com', 'Angie.Masotti@c...","{'Manisha.Joshi@cantire.com', 'Dilakshika.Kuga...","{'[EXTERNAL] - Euro-Pro | 1365 Cost Down', '[E...","{'2022/03/01 18:46:23', '2022/05/12 15:33:39',..."
1,629ac74d1f717403516b0852e43461ad,{'jennifervassallo@gorillaglue.com'},{'andrew.holder@cantire.com'},{'[EXTERNAL] - FW: Deal 0828-2022 Cost Down - ...,{'2022/01/26 19:56:17'}
2,9a58f66ce99522ba8649112a143027e0,{'Andrew.Holder@cantire.com'},{'Dipesh.Mistry@cantire.com'},{'[EXTERNAL] - Deal 0876-2022 Cost Downs'},{'2022/05/19 19:45:22'}
3,79baab175907a5c9525471edf8c298b8,{'Andrew.Holder@cantire.com'},{'Dipesh.Mistry@cantire.com'},{'[EXTERNAL] - Deal 0876-2022 Cost Downs'},{'2022/05/19 19:45:22'}
4,962a4f55273c5358bb5100e64b9d2df5,{'Andrew.Holder@cantire.com'},{'Dipesh.Mistry@cantire.com'},{'[EXTERNAL] - Deal 0876-2022 Cost Downs'},{'2022/05/19 19:45:22'}
...,...,...,...,...,...
4577,56039898dd0baa9c720b76983ed3097c,"{'kspence@wd40.com', 'Gregory.Jennings@cantire...","{'Gregory.Jennings@cantire.com', 'gregory.jenn...","{'RE: [EXTERNAL] WD40 Cost Down - D330', 'FW: ...","{'2022/05/27 14:35:19', '2022/05/27 14:34:13'}"
4578,b88f24c4c8d8c0b665c8ad945a602941,{'kspence@wd40.com'},"{'Samara.Sayeed@cantire.com', 'Yvonne.Ng@canti...",{'RE: [EXTERNAL] RE: WD-40: Q4 Cost Down Agree...,{'2021/10/20 19:02:49'}
4579,ad912bda79550d4fca64331606ace7be,{'Gregory.Jennings@cantire.com'},{'kspence@wd40.com'},{'[EXTERNAL] - WD40 Cost Down - D330'},{'2022/05/26 17:30:54'}
4580,1fe45e6333f07ea85abe67ae5762a66e,{'Gregory.Jennings@cantire.com'},{'gregory.jennings.92@gmail.com'},{'FW: [EXTERNAL] D1118 In-Spring Guide - WD-40...,{'2022/02/28 20:35:42'}


In [22]:
print(len(df_fin["DATE"]))

4582


In [23]:
import re
lis = []
for i in range(len(df_fin["DATE"])):
    date = df_fin["DATE"][i]
    day = date.split(",")
    year = set()
    if len(day)>1:
        for val in day:
            m_y = re.search(r'(\d{4})/(\d{2})', val)
            if m_y:
                year.add(m_y.group())
    else:
        m_y = re.search(r'(\d{4})/(\d{2})',date)
        if m_y:
            year.add(m_y.group())
    lis.append(year)
print(len(lis))

4582


In [24]:
df_fin[:]["DATE"].update(lis)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_fin[:]["DATE"][:10]

0    {2022/05, 2022/03}
1             {2022/01}
2             {2022/05}
3             {2022/05}
4             {2022/05}
5             {2022/05}
6             {2022/05}
7             {2022/05}
8             {2022/05}
9             {2022/05}
Name: DATE, dtype: object

In [26]:
df_fin.head(2)

,MD5,FROM,TO,SUBJECT,DATE
0,a7551870dbe26c7b810c986c023f7ea6,"{'Manisha.Joshi@cantire.com', 'Angie.Masotti@c...","{'Manisha.Joshi@cantire.com', 'Dilakshika.Kuga...","{'[EXTERNAL] - Euro-Pro | 1365 Cost Down', '[E...","{2022/05, 2022/03}"
1,629ac74d1f717403516b0852e43461ad,{'jennifervassallo@gorillaglue.com'},{'andrew.holder@cantire.com'},{'[EXTERNAL] - FW: Deal 0828-2022 Cost Down - ...,{2022/01}


In [27]:
df_fin.to_csv("new_MD5_Costdown.csv")

In [49]:
body = {
    "size":100,
    "query":{
        "match_phrase":{
            "MD5":"655c817d5ba682609674a11716e1a8cd"
        }
    }
}
res = es.search(index="*",body=body)
res["hits"]["hits"]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if __name__ == '__main__':


[]